This notebook reads in image data, creates labels, and runs a modified VGG network to classify the images. It outputs a .mat file with the validation accuracy, validation loss, training accuracy, and training loss.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from matplotlib import pyplot as plt

In [ ]:
file = 'INSERT_PATH_TO_UNIFORM_IMAGES_FILE.npz'
loaded = np.load(file)
uniform_img = loaded['data']
del loaded

In [ ]:
file = 'INSERT_PATH_TO_LESION_IMAGES_FILE.npz'
loaded = np.load(file)
lesion_img = loaded['data']
del loaded

In [ ]:
data = np.dstack((uniform_img, lesion_img))
data = np.moveaxis(data, 2, 0)
del lesion_img
del uniform_img

In [1]:
#new_labels = np.concatenate((labels, np.transpose(labels)))
new_labels = np.zeros((2000, 2))
new_labels[0:1000, 0] = 0
new_labels[0:1000, 1] = 1
new_labels[1000:2000, 0] = 1
new_labels[1000:2000, 1] = 0

NameError: name 'np' is not defined

In [ ]:
X_train, x_test, y_train, y_test = train_test_split(data, new_labels, test_size = 0.10, random_state=42, shuffle=True)
X_train = X_train.reshape(X_train.shape[0], 390, 390, 1)
x_test = x_test.reshape(x_test.shape[0], 390, 390, 1)


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(8, (3,3), padding='same', activation='relu', input_shape=(390, 390, 1)),

    tf.keras.layers.MaxPool2D(padding='same'),
    
    tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu'),

    tf.keras.layers.MaxPool2D(padding='same'),

    tf.keras.layers.Conv2D(256, (3,3), padding='same', activation='relu'),
    tf.keras.layers.Conv2D(256, (3,3), padding='same', activation='relu'),
    
    tf.keras.layers.MaxPool2D(padding='same'),
    
    tf.keras.layers.Conv2D(512, (3,3), padding='same', activation='relu'),
    tf.keras.layers.Conv2D(512, (3,3), padding='same', activation='relu'),
    
    tf.keras.layers.MaxPool2D(padding='same'),
    
    tf.keras.layers.Conv2D(512, (3,3), padding='same', activation='relu'),
    tf.keras.layers.Conv2D(512, (3,3), padding='same', activation='relu'),
        
    tf.keras.layers.MaxPool2D(padding='same'),
    
    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(2, activation='softmax')
])

In [ ]:
model.compile(optimizer=tf.train.AdamOptimizer(0.00001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
hist = model.fit(X_train, y_train, epochs= 15, validation_data = (x_test, y_test),
          verbose=1)

In [ ]:
val_acc = hist.history['val_acc']
train_loss = hist.history['loss']
val_loss = hist.history['val_loss']
train_acc = hist.history['acc']

In [ ]:

scipy.io.savemat('XMHz_YdB.mat', mdict={'val_acc':val_acc, 'train_loss':train_loss, 'val_loss':val_loss, 'train_acc':train_acc})

In [ ]:
# Optional Code for Visualizing Images
image_num = 35
test_img = np.squeeze(X_train[image_num, :, :])
plt.imshow(test_img, cmap='gray')
y_train[image_num]